# Using the Bailo module 

To connect to the API, you will need to authenticate the Bailo client.  


For Cognito authentication:
```
from bailoclient import Bailo, BailoConfig, CognitoConfig

auth = CognitoConfig(
    username="username",
    password="password",
    user_pool_id="user-pool-id",
    client_id="client-id",
    client_secret="secret",
    region="region",
)
bailo = Bailo(
    config=BailoConfig(
        auth=auth,
        bailo_url="https://bailo.io",
        ca_cert="path/to/ca",
    )
)
```

For PKI authentication:
```
from bailoclient import Bailo, BailoConfig, Pkcs12Config

auth = Pkcs12Config(
    pkcs12_filename="path/to/file.pem",
    pkcs12_password="password"
)
bailo = Bailo(
    config=BailoConfig(
        auth=auth,
        bailo_url="https://bailo.io",
        ca_cert="path/to/ca",
    )
)
```


If you don't need a client and just want to make use of model loading/bundling functionality:

```
from bailoclient import Bailo, BailoConfig, Pkcs12Config

bailo = Bailo(
    config=BailoConfig(auth=None, bailo_url="none")
)
```


# Example workflow

This example workflow will demonstrate the capability of the Python client for interacting with Bailo, as well as its automated model bundling and loading functionality.

It is assumed that you have a valid .env file configured (see the README for more information).

You will need to have sklearn installed in your Python environment to generate the test model. 

In [ ]:
# !pip install -U scikit-learn

#### Create a simple model

We will use sklearn to create a basic model trained on the Iris dataset. The model returns predictions as a numeric label.

In [ ]:
from sklearn import datasets
from sklearn.svm import SVC

iris = datasets.load_iris()
clf = SVC()
clf.fit(iris.data, iris.target)

#### Bundle the required model files

Bailo's model bundling functionality can handle much of the process for you. All you need to provide is the model 'flavour' (i.e. the package that the model was produced with)

* Generates the requirements.txt file based on an input directory/module/notebook file
* Gets the corresponding model.py template (**warning** most of these are currently untested)
* Saves your model (many of the bundler flavours available make use of MLflow to save models - you may need this installed to make use of some of the bundler functionality)
* Organises your files into code and binary zip folders

The sklearn bundler does not require MLflow and will export your model to a .pkl file for you.

In [ ]:
from bailoclient import Bailo, Pkcs12Config, BailoConfig

bailo_url = "..."
auth = Pkcs12Config(...)
bailo = Bailo(config=BailoConfig(auth=auth, bailo_url=bailo_url))

To check what flavours are available for bundling/loading:

In [ ]:
bailo.flavours

In [ ]:
output_path = "./sklearn_example"

bailo.bundle_model(model=clf, output_path=output_path, model_flavour="sklearn")

#### Upload the bundled model files

To upload the model, we need to provide metadata, or the model card. 

There is a minimal amount of metadata that must be provided that you can look at via the Bailo client. We will set a name, description and overview for our model before uploading it via the client.

In [ ]:
model_binary = f"{output_path}/binary.zip"
model_code = f"{output_path}/code.zip"

# set some of the metadata for the model
model_metadata = bailo.minimal_metadata
model_metadata["highLevelDetails"]["name"] = "sklearn model"
model_metadata["highLevelDetails"]["modelInASentence"] = "predicts iris data"
model_metadata["highLevelDetails"][
    "modelOverview"
] = "sklearn model to predict iris data"

# upload the model
uploaded_model = bailo.upload_model(
    metadata=model_metadata,
    binary_file=model_binary,
    code_file=model_code,
)

#### Check the model has been uploaded

Use the client to retrieve all of your models - you should have a model called sklearn-model-xxxxxx

In [ ]:
user_models = bailo.get_my_models()

for model in user_models:
    print(model.uuid)

#### Update the model with a new version of the binary

As your model is developed over time you will want to upload new versions. 

In this case, we want to improve our model by having it return strings of the actual classification labels instead of just a numeric value for its predictions.

We will update the model to do this:

In [ ]:
clf = SVC()

clf.fit(iris.data, iris.target_names[iris.target])

When we have the updated model we can zip the binary file. We'll skip the Bailo bundling step because we haven't updated any of the code files. Instead we'll create a new binary zip file with our new model pkl file

In [ ]:
import pickle
import zipfile
import os

os.makedirs(f"{output_path}/new_binary", exist_ok=True)

with open(f"{output_path}/new_binary/model.pkl", "wb") as f:
    pickle.dump(clf, f)

zipfile.ZipFile(f"{output_path}/new_binary/binary.zip", mode="w").write(
    f"{output_path}/new_binary/model.pkl", arcname="model.pkl"
)

#### Update the model metadata

We can get the metadata from the model card we retrieved when we called bailo.get_my_models(). We can look at the metadata by calling the display function.

You may also want to see which fields are accessible in the model card:

```
    dir(model_card)
```

Or to look at the validation schema for the model (which should give an indication of which metadata fields are expected and what they should look like):
```
    schema = bailo.get_model_schema(model_uuid)
```

In [ ]:
model_card = user_models[0]

model_card.display()

#### Update the required fields on the model card

As a minimum you will need to update the model version number

In [ ]:
# get model and required fields for updating (UUID and metadata)
model_card = user_models[0]
model_uuid = model_card.uuid
metadata = model_card.latestVersion.metadata


# update some of the metadata fields for the new model version
metadata.highLevelDetails.name = f"Updated sklearn model"
metadata.highLevelDetails.modelCardVersion = "v2.0"

#### Validate the model card

Check that all the fields that we have provided are valid input

In [ ]:
result = model_card.validate()
for error in result.errors:
    print(f"{error.field}: {error.description}")

#### Push the new model version up to Bailo

In [ ]:
update_resp = bailo.update_model(
    metadata=metadata,
    model_uuid=model_uuid,
    binary_file=f"{output_path}/new_binary/binary.zip",
    code_file=f"{output_path}/code.zip",  # code is unchanged, but must still be provided
)

#### Check that the model has been updated

Get a new list of user models and identify the most recently uploaded model. Check that the latest version of this model has the expected name

In [ ]:
user_models = bailo.get_models()
latest_model = user_models[0]

latest_model.latestVersion.metadata.highLevelDetails.name

#### Request a deployment

To request a deployment, we have to provide metadata relating to the deployment request.

You can access the minimal deployment metadata on the Bailo interface in the same way as we accessed the minimal model metadata. 

In [ ]:
deployment_metadata = bailo.minimal_deployment_metadata
deployment_metadata

Let's edit this metadata to request a deployment for our model

* Give the deployment a name
* Give the model an end date of tomorrow (end date is not a required field)
* Link the deployment request to the model we are requesting via the model UUID
* Set the owner to be the current user

In [ ]:
import datetime

# name the deployment
deployment_metadata["highLevelDetails"]["name"] = "sklearn test deployment"

# set end date to tomorrow
end_date = str(datetime.date.today() + datetime.timedelta(days=1))
deployment_metadata["highLevelDetails"]["endDate"]["date"] = end_date

# set model ID to our new model's uuid
deployment_metadata["highLevelDetails"]["modelID"] = model_uuid

# set owner to current user id
deployment_metadata["contacts"]["owner"][0]["id"] = bailo.get_me().id

deployment_metadata

In [ ]:
deployment_request = bailo.request_deployment(deployment_metadata)

#### Download the model files for a deployment

With an **approved** deployment you can request to download the model files. 

To carry out this step you will need your deployment request to be approved. Deployment requests cannot be approved via the Python module. 

By default the download_model_files function will download both the model binary and code. To specify either binary or code download, use e.g. file_type='binary'

In [ ]:
deployment_uuid = deployment_request["uuid"]
model_version = latest_model.latestVersion.metadata.highLevelDetails.modelCardVersion

bailo.download_model_files(
    deployment_uuid=deployment_uuid,
    model_version=model_version,
    output_dir=f"{output_path}/downloaded_model",
)

#### Load the model into memory

With the model files downloaded we can use the sklearn loader function to automatically load the model object into memory - we now have a sklearn SVC model loaded. 

In [ ]:
loaded_model = bailo.load_model(
    f"{output_path}/downloaded_model/model.pkl", model_flavour="sklearn"
)
type(loaded_model)

#### Run some predictions

Finally, we can run some predictions on the model and see that the model binary has been updated to return the labels of the predictions. 

In [ ]:
loaded_model.predict(iris.data)